In [ ]:
#https://github.com/hananedupouy/LLMs-in-Finance/blob/main/RAG/llamaindex/LlamaParse_GPT_4o_mode_10K_report.ipynb

In [5]:
import requests
import law_api

def get_wikipedia_page(page_title):
    base_url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "prop": "extracts",
        "format": "json",
        # "exintro": True,
        "titles": page_title,
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    page_id = list(data["query"]["pages"].keys())[0]
    extract = data["query"]["pages"][page_id]["extract"]
    return extract

# Usage:
#print(get_wikipedia_page("Python (programming language)"))


In [ ]:
case = input("please select a supreme court decision that you would like to talk about")

print(get_wikipedia_page(case))

In [8]:
info = get_wikipedia_page(case)

In [9]:
print(info)

<p class="mw-empty-elt">



</p>

<p><i><b>Brown v. Board of Education of Topeka</b></i>, 347 U.S. 483 (1954), was a landmark decision of the U.S. Supreme Court ruling that U.S. state laws establishing racial segregation in public schools are unconstitutional, even if the segregated schools are otherwise equal in quality. The decision partially overruled the Court's 1896 decision <i>Plessy v. Ferguson</i>, which had held that racial segregation laws did not violate the U.S. Constitution as long as the facilities for each race were equal in quality, a doctrine that had come to be known as "separate but equal". The Court's unanimous decision in <i>Brown</i>, and its related cases, paved the way for integration and was a major victory of the civil rights movement, and a model for many future impact litigation cases.
</p><p>The case began in 1951 when the public school system in Topeka, Kansas, refused to enroll local black resident Oliver Brown's daughter at the school closest to their ho

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
LLAMAPARSE_API_KEY = userdata.get('LLAMAPARSE_API_KEY')
ANTHROPIC_API_KEY = userdata.get("CLAUDE_API_KEY")

In [ ]:

import nest_asyncio
nest_asyncio.apply()

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
LLAMAPARSE_API_KEY = userdata.get('LLAMAPARSE_API_KEY')
ANTHROPIC_API_KEY = userdata.get("CLAUDE_API_KEY")

In [ ]:
# from llama_parse import LlamaParse
from llama_index.llms.anthropic import Anthropic
from llama_index.core import Settings

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import nest_asyncio;
nest_asyncio.apply()

pdf_name = "amzn_2023_10k.pdf"
# use SimpleDirectoryReader to parse our file
documents = SimpleDirectoryReader(input_files=[pdf_name]).load_data()
from llama_index.core import Settings

embed_model = "local:BAAI/bge-small-en-v1.5" #https://huggingface.co/collections/BAAI/bge-66797a74476eb1f085c7446d

vector_index_std = VectorStoreIndex(documents, embed_model = embed_model)

from llama_index.llms.openai import OpenAI

llm_gpt35 = OpenAI(model="gpt-3.5-turbo", api_key = OPENAI_API_KEY)
query_engine_gpt35 = vector_index_std.as_query_engine(similarity_top_k=3, llm=llm_gpt35)

llm_gpt4o = OpenAI(model="gpt-4o-mini", api_key = OPENAI_API_KEY)
query_engine_gpt4o = vector_index_std.as_query_engine(similarity_top_k=3, llm=llm_gpt4o)




tokenizer = Anthropic().tokenizer
Settings.tokenizer = tokenizer

llm_claude = Anthropic(model="claude-3-5-sonnet-20240620", api_key=ANTHROPIC_API_KEY)
query_engine_claude = vector_index_std.as_query_engine(similarity_top_k=3, llm=llm_claude)

In [ ]:
def search_case(case_name):
	query_case_num = "What is the case number"
	response = query_engine_gpt4o.query(query_case_num)
	print("\nGPT-4o")
	print(str(response))
    
print(search_case(case_name))

In [ ]:
query1 = "What was the net income in 2023?"
response = query_engine_gpt35.query(query1)
print("GPT-3.5 Turbo")
print(str(response))

resp = query_engine_gpt4o.query(query1)
print("\nGPT-4o")
print(str(resp))

resp = query_engine_claude.query(query1)
print("\nClaude 3.5 Sonnet")
print(str(resp))

# print(resp.source_nodes[0].get_content()) # to get the source_node used to answer